In [1]:
import pandas as pd
import numpy as np
#import sweetviz as sv
import os
import plotly.express as px
import seaborn as sns
import matplotlib as mlp
import psycopg2
import plotly.offline as pyo
import random
from dash import Dash, html, dcc, Input, Output
import dash_daq as daq
from flask import Flask
from gevent.pywsgi import WSGIServer
import os, signal

In [2]:
def connection(sql):
    con = psycopg2.connect(host='35.247.202.234', database='dw',
    user='consulta', password='teste123')
    cur = con.cursor()    
    cur.execute(sql)
    data = cur.fetchall()
    cols = []
    for elt in cur.description:
        cols.append(elt[0])

    df = pd.DataFrame(data=data,columns=cols)
    con.close();
    return df

In [3]:
query = """select * from public.dimensoes_est de
inner join escolas.aluno a 
on de.id_aluno = a.id_aluno 
inner join escolas.turma t 
on a.id_turma = t.id_turma 
inner join escolas.escola e 
on t.id_escola = e.cod_escola"""
df = connection(query)


In [4]:
df = df.loc[:,~df.columns.duplicated()].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 46 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   id                          62 non-null     int64              
 1   id_aluno                    62 non-null     int64              
 2   E_ESCV                      62 non-null     float64            
 3   E_PROFV                     62 non-null     float64            
 4   E_FAMV                      62 non-null     float64            
 5   E_COMV                      62 non-null     float64            
 6   E_ESTV                      62 non-null     float64            
 7   E_ESCC                      62 non-null     object             
 8   E_PROFC                     62 non-null     object             
 9   E_FAMC                      62 non-null     object             
 10  E_COMC                      62 non-null     object             


In [5]:
cross_tab_prop = df[['id_aluno','nome_aluno','id_turma','nome_turma','cod_escola','escola','E_ESCV', 'E_PROFV', 'E_FAMV', 'E_COMV', 'E_ESTV','E_ESCC', 'E_PROFC', 'E_FAMC', 'E_COMC', 'E_ESTC']]

cross_tab_prop.set_index('id_aluno')
cross_tab_prop

,id_aluno,nome_aluno,id_turma,nome_turma,cod_escola,escola,E_ESCV,E_PROFV,E_FAMV,E_COMV,E_ESTV,E_ESCC,E_PROFC,E_FAMC,E_COMC,E_ESTC
0,14799,ADRYAN MAGNO DA SILVA NEVES,695,None,11000937,EEEFM DR OSWALDO PIANNA,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1,1,1
1,19175,Monique Gomes de Souza,883,None,15070387,E M E F PROFESSORA DALILA LEAO,2.000000,1.000000,2.000000,1.000000,1.777778,1,1,1,1,1
2,19174,Elielson Rodrigues Viana,892,None,15070387,E M E F PROFESSORA DALILA LEAO,2.500000,1.000000,3.500000,1.888889,2.888889,1,1,2,1,2
3,19152,Jeferson Oliveira dos Santos,1064,None,15070387,E M E F PROFESSORA DALILA LEAO,2.000000,1.500000,3.333333,1.888889,2.888889,1,1,1,1,2
4,19150,Hudson Andrade Belo,1064,None,15070387,E M E F PROFESSORA DALILA LEAO,4.000000,1.000000,1.000000,1.000000,1.666667,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,19176,ANDRIA RODRIGUES DE ARAGAO,1067,None,15562158,E M E F PROFESSORA DINORA TAVARES GONCALVES,3.166667,1.000000,2.666667,1.333333,3.444444,2,1,1,1,3
58,19010,MARIA EDUARDA PIMENTEL DO NASCIMENTO,1061,None,15562158,E M E F PROFESSORA DINORA TAVARES GONCALVES,2.333333,3.000000,4.166667,2.777778,3.555556,1,2,2,2,3
59,18999,GILVANA DOS SANTOS CARVALHO,1061,None,15562158,E M E F PROFESSORA DINORA TAVARES GONCALVES,1.000000,3.000000,3.833333,1.666667,2.888889,1,2,2,1,2
60,18994,EDIELEN LUIZA PRAZERES DA GAMA,1061,None,15562158,E M E F PROFESSORA DINORA TAVARES GONCALVES,1.000000,2.333333,4.333333,3.222222,4.666667,1,1,3,2,3


In [7]:
filter = cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO'
escola_dimensao = cross_tab_prop[filter]
E_ESCV = np.mean(escola_dimensao['E_ESCV'])
E_PROFV = np.mean(escola_dimensao['E_PROFV'])
E_FAMV = np.mean(escola_dimensao['E_FAMV'])
E_COMV = np.mean(escola_dimensao['E_COMV'])
E_ESTV = np.mean(escola_dimensao['E_ESTV'])

In [8]:
filter = (cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO') & (cross_tab_prop['id_turma']==1064)
turma_dimensao = cross_tab_prop[filter]
E_ESCV = np.mean(turma_dimensao['E_ESCV'])
E_PROFV = np.mean(turma_dimensao['E_PROFV'])
E_FAMV = np.mean(turma_dimensao['E_FAMV'])
E_COMV = np.mean(turma_dimensao['E_COMV'])
E_ESTV = np.mean(turma_dimensao['E_ESTV'])

In [27]:
filter = (cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO') & (cross_tab_prop['id_turma']==1064) & (cross_tab_prop['nome_aluno']=='Jeferson Oliveira dos Santos')
aluno_dimensao = cross_tab_prop[filter]
E_ESCV = np.mean(aluno_dimensao['E_ESCV'])
E_PROFV = np.mean(aluno_dimensao['E_PROFV'])
E_FAMV = np.mean(aluno_dimensao['E_FAMV'])
E_COMV = np.mean(aluno_dimensao['E_COMV'])
E_ESTV = np.mean(aluno_dimensao['E_ESTV'])
#aluno_dimensao['E_ESCV']

In [ ]:
app = Dash(__name__)
#app = Flask(__name__)

DIV_STYLE = {
  "height": "900px",
  "display": "flex",
  "justify-content": "center",
  "align-items": "center",
}

CHART_BOX_STYLE = {
  "display": "flex",
  "justify-content": "center",
  "flex-wrap": "wrap",
  "align-items": "center",
}

CHART_STYLE = {
  "flex": "0 0 33.33%",
  "maxWidth": "33.33%",
}

app.layout = html.Div(
  [
    html.Div(
      [
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.83],"yellow":[2.83,4],"red":[4,7]}},
            value=E_ESCV,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E-ESC',
            max=7,
            min=0,
          ),
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.66],"yellow":[2.66,3.66],"red":[3.66,7]}},
            value=E_PROFV,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E-PROF',
            max=7,
            min=0,
          ),
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,3.33],"yellow":[3.33,4.33],"red":[4.33,7]}},
            value=E_FAMV,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E-FAM',
            max=7,
            min=0,
          ),   
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.44],"yellow":[2.44,3.55],"red":[3.55,7]}},
            value=E_COMV,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E-COM',
            max=7,
            min=0,
          ),
          
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.44],"yellow":[2.44,3.41],"red":[3.41,7]}},
            value=E_ESTV,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E-EST',
            max=7,
            min=0,
          ),
      ],
      style=CHART_STYLE),
      ],
      style=CHART_BOX_STYLE,
    )
  ],
  style=DIV_STYLE,  
)

if __name__ == '__main__':
    app.run()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [06/Dec/2022 18:17:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 18:17:58] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_0m1667509735.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 18:17:58] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_0m1667509735.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 18:17:58] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_7_0m1667509735.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 18:17:58] "GET /_dash-component-suites/dash_daq/dash_daq.v0_5_0m1596869724.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 18:17:58] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_7_0m1667509735.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 18:17:58] "GET /_dash-component-suites/dash/dash_table/bundle.v5_2_0m1667509735.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 18:17:58] "GET /_dash-component-suites/dash/html/dash_html